# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [15]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [16]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [17]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
PRICE_DATA


parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
#parquet_files[:5]
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px


<dask_expr.expr.DataFrame: expr=ReadParquet(3e7ffa9)>

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [18]:
# Write your code below.
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(    
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))


C:\Users\dugga\AppData\Local\Temp\ipykernel_3400\777055813.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [19]:
dd_feat

<dask_expr.expr.DataFrame: expr=Assign(frame=Assign(frame=Assign(frame=Assign(frame=Assign(frame=Assign(frame=GroupByApply(frame=ReadParquet(3e7ffa9), observed=False, group_keys=False, func=<function <lambda> at 0x00000240E0D69A60>, meta=_NoDefault.no_default, args=(), kwargs={})))))))>

In [20]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
GRMN,2009-01-02,19.000000,22.000000,19.000000,21.770000,12.750558,1884800,Consumer Discretionary,Consumer Electronics,2009,NaN,NaN,3.000000
GRMN,2009-01-05,20.830000,22.040001,20.750000,22.000000,12.885260,1897500,Consumer Discretionary,Consumer Electronics,2009,21.770000,0.010565,1.290001
GRMN,2009-01-06,20.500000,21.250000,19.700001,21.090000,12.352277,3074400,Consumer Discretionary,Consumer Electronics,2009,22.000000,-0.041364,1.549999
GRMN,2009-01-07,20.610001,21.090000,20.200001,21.030001,12.317138,1432700,Consumer Discretionary,Consumer Electronics,2009,21.090000,-0.002845,0.889999
GRMN,2009-01-08,21.000000,21.350000,20.170000,21.030001,12.317138,1206500,Consumer Discretionary,Consumer Electronics,2009,21.030001,0.000000,1.180000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GILD,2018-12-24,62.470001,62.830002,60.470001,60.540001,48.376888,6162600,Health Care,Biotechnology,2018,62.619999,-0.033216,2.360001
GILD,2018-12-26,60.650002,63.230000,60.320000,63.220001,50.518440,8824400,Health Care,Biotechnology,2018,60.540001,0.044268,2.910000
GILD,2018-12-27,62.400002,63.020000,60.470001,62.939999,50.294697,9548200,Health Care,Biotechnology,2018,63.220001,-0.004429,2.549999


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [21]:
# Write your code below.

dd_feat = dd_feat.compute()
dd_feat['10dayrollingaverage'] = dd_feat['returns'].rolling(10).mean()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.